In [ ]:
# hide

from pprint import pprint

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp persistence

<IPython.core.display.Javascript object>

In [ ]:
import json
import sqlite3

from will_it_saturate.core import BaseRepository


class SqliteRepository(BaseRepository):
    connection: sqlite3.Connection

    class Config:
        arbitrary_types_allowed = True

    def execute_stmt(self, stmt, ignore_error=False):
        cursor = self.connection.cursor()
        try:
            cursor.execute(stmt)
        except sqlite3.OperationalError as exc:
            if not ignore_error:
                raise (exc)

    def create_benchmark_table(self):
        stmt = """
            create table benchmark (
                benchmark_id INTEGER PRIMARY KEY,
                created DATE DEFAULT (datetime('now','localtime')),
                data text not NULL UNIQUE
            )
        """
        self.execute_stmt(stmt, ignore_error=True)

    def create_result_table(self):
        stmt = """
            create table result (
                result_id INTEGER PRIMARY KEY,
                benchmark_id integer not NULL,
                created DATE DEFAULT (datetime('now','localtime')),
                server text not NULL,
                client text not NULL,
                file_size integer not NULL,
                complete_size integer not NULL,
                elapsed real not NULL,
                FOREIGN KEY(benchmark_id) REFERENCES benchmark(benchmark_id)
            )
        """
        self.execute_stmt(stmt, ignore_error=True)

    def create_tables(self):
        self.create_benchmark_table()
        self.create_result_table()

    def add_benchmark(self, benchmark):
        stmt = """
            insert into benchmark (data)
            values (?)
        """
        cursor = self.connection.cursor()
        data = benchmark.json()
        try:
            cursor = cursor.execute(stmt, [data])
        except sqlite3.IntegrityError as e:
            pass
        self.connection.commit()

    def get_benchmark(self, benchmark):
        stmt = """
            select *
              from benchmark
             where data=?
        """
        cursor = self.connection.cursor()
        cursor.execute(stmt, [benchmark.json()])
        print("fetchall: ")
        return cursor.fetchone()

    def get_benchmark_id(self, benchmark):
        row = self.get_benchmark(benchmark)
        return row[0]

    def add_result(self, benchmark, result):
        benchmark_id = self.get_benchmark_id(benchmark)
        columns = [
            "benchmark_id",
            "server",
            "client",
            "file_size",
            "complete_size",
            "elapsed",
        ]
        columns_str = ",".join(columns)
        values_str = ",".join(["?" for c in columns])
        stmt = f"""
            insert into result ({columns_str})
            values ({values_str})
        """
        print(stmt)
        cursor = self.connection.cursor()
        result_dict = result.dict()
        result_dict["benchmark_id"] = benchmark_id
        values = [result_dict[c] for c in columns]
        try:
            cursor = cursor.execute(stmt, values)
        except sqlite3.IntegrityError as e:
            pass
        self.connection.commit()

    def persist(self, benchmark):
        persisted_results = self.results.get(benchmark, {})
        for result in benchmark.results:
            if result not in persisted_results:
                persisted_results[result] = result
        self.results[benchmark] = persisted_results

<IPython.core.display.Javascript object>

# Usage

In [ ]:
conn = sqlite3.connect("example.db")
repo = SqliteRepository(connection=conn)
repo.create_tables()

<IPython.core.display.Javascript object>

In [ ]:
from will_it_saturate.core import Benchmark

benchmark = Benchmark()
benchmark = Benchmark()
repo.add_benchmark(benchmark)
repo.add_benchmark(benchmark)  # should not raise integrity exception

<IPython.core.display.Javascript object>

In [ ]:
benchmark.json()

'{"duration": 30, "bandwidth": 125000000, "file_sizes": [10000000, 1000000, 100000], "rows": [], "file_creator": {}, "platform": "x86_64", "cpuinfo": {"python_version": "3.9.2.final.0 (64 bit)", "cpuinfo_version": [7, 0, 0], "cpuinfo_version_string": "7.0.0", "arch": "X86_64", "bits": 64, "count": 12, "arch_string_raw": "x86_64", "vendor_id_raw": "GenuineIntel", "brand_raw": "Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz", "hz_advertised_friendly": "2.6000 GHz", "hz_actual_friendly": "2.6000 GHz", "hz_advertised": [2600000000, 0], "hz_actual": [2600000000, 0], "l2_cache_size": 262144, "stepping": 10, "model": 158, "family": 6, "flags": ["1gbpage", "3dnowprefetch", "abm", "acpi", "adx", "aes", "apic", "avx", "avx1.0", "avx2", "bmi1", "bmi2", "clflush", "clflushopt", "clfsh", "clfsopt", "cmov", "cx16", "cx8", "de", "ds", "ds_cpl", "dscpl", "dtes64", "dts", "em64t", "erms", "est", "f16c", "fma", "fpu", "fpu_csds", "fxsr", "ht", "htt", "ibrs", "intel_pt", "invpcid", "ipt", "l1df", "lahf", "lahf

<IPython.core.display.Javascript object>

In [ ]:
repo.get_benchmark(benchmark)[0]

fetchall: 


1

<IPython.core.display.Javascript object>

In [ ]:
from will_it_saturate.core import BenchmarkResult

file_size = 10 ** 6
complete_size = 100 * file_size
result = BenchmarkResult(
    server="nginx",
    client="httpx",
    file_size=file_size,
    elapsed=3.0,
    complete_size=complete_size,
)
print(result.dict())

{'server': 'nginx', 'client': 'httpx', 'file_size': 1000000, 'elapsed': 3.0, 'complete_size': 100000000, 'file_size_h': '976.56KB', 'bytes_per_second': 33333333.333333332, 'bytes_per_second_h': '31.79MB'}


<IPython.core.display.Javascript object>

In [ ]:
repo.add_result(benchmark, result)

fetchall: 

            insert into result (benchmark_id,server,client,file_size,complete_size,elapsed)
            values (?,?,?,?,?,?)
        


<IPython.core.display.Javascript object>